Getting Genre data can be done by web scraping https://www.chosic.com/music-genre-finder/

    I can make a table with 3 columns [song, artist, genre] so that a song will have multiple rows
    From here I can take a word count to see the top genres and then group each song into larger genres so that one song 
    will have 1 genre, with a misc to catch-all
    After I am satisfied of the amount of misc ranked songs, I append it to the table I've got below
   
# Next steps:
    Add in a filter so I only sesarch songs that have not been previously searched (File: "Data - Scrapped Genres")

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

import os

In [2]:
def unique_element(parent, by, value ):
    elements = parent.find_elements(by, value)
    if len(elements) == 0:
        print("No elements found from " + str(parent))
        return
    if len(elements) > 1:
        print("Non-unique element with criteria")
        try:
            name_cutoff = parent.get_property("outerHTML").find(">") + 1
        except:
            print("Element does not have .get_property()")
        else:
            print("|Origin: ", parent.get_property("outerHTML")[0:name_cutoff].strip())
        print("|    By: ", by)
        print("| Value: ", value)        
    
    return elements[0]

def get_element_name(element):
    if element is None:
        return ""
    text = element.get_property("outerHTML")
    if text is None:
        return ""
    
    try:
        name_cutoff = text.find(">") + 1
    except:
        return text.strip()
    finally:
        return text[0:name_cutoff].strip()
    
def element_properties(element):
    print("**Properties**")
    print("| Element: ", get_element_name(element))
    #print("|      ID: ", element.get_property("id"))
    #print("|Attributes: ", element.get_property("attributes"))
    print("|  Parent: ", get_element_name(element.get_property("offsetParent")))
    children = element.get_property("children")
    print("|Children: (", len(children), ")", sep="")
    for child in element.get_property("children"):
          print("||       ", get_element_name(child))
    print("|Previous Sibling: ", get_element_name(element.get_property("previousElementSibling")))
    print("|    Next Sibling: ", get_element_name(element.get_property("nextElementSibling")))
    
def site_load():
    """Pauses the program until the website is loaded"""
    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "/html/body"))
        )
    except:
        raise ValueError("Website Not Loading")

In [3]:
os.chdir("C:\\Users\\ryane\\Anaconda Work")
driver = webdriver.Chrome()
driver.get("https://www.chosic.com/music-genre-finder/")
#driver.maximize_window()

# Code to loop through all songs

In [4]:
def refresh():
    for i in np.arange(max_len):
        search_bar.send_keys("\b\b\b\b\b")
    option = driver.find_element(By.ID, "suggestion-options")
    option.click()
    
    to_artist = driver.find_element(By.ID, "artist")
    to_artist.click()
    
    option.click()
    
    to_song = driver.find_element(By.ID, "song")
    to_song.click()

In [13]:
os.chdir("C:\\Users\\ryane\\Anaconda Work\\Spotify")

# Songs will contain a list of songs and artists
songs = pd.read_json("./Data - 2020/StreamingHistory0.json")
songs = songs[["artistName", "trackName"]]
songs.rename(columns={"artistName":"artist", "trackName":"track"}, inplace=True)
songs = songs.drop_duplicates()

# Categ is being created to find out the genres of each song
categ = pd.DataFrame(columns=['artist', 'track',"genre"])
categ

song_stats = pd.DataFrame(columns=["artist", "track", "Popularity", "Happiness", "Danceability", "Energy",
                                   "Acousticness", "Instrumentalness", "Liveness", "Speechiness", "Length",
                                   "Tempo", "Loudness"])

# issue is going to be a df of all indexes of songs that do not get all information
#issue = pd.DataFrame(columns=["Index", "artist", "track", "Error Code"])
issue = pd.read_csv("Scraping Errors.csv")

max_len = np.ceil((max(songs["artist"].apply(len)) + max(songs["track"].apply(len)))/5)

In [14]:
existing = pd.read_csv("Data - Scraped Genres.csv")
existing = existing[["artist", "track"]].drop_duplicates()

In [15]:
def is_genres():
    # Returns true if no issues, false otherwise
    try:
        genre_parent = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='spotify-result']/div[@class='pl-tags tagcloud']"))
        )
    except:
        return False
    return True

In [16]:
# Version to edit
driver.get("https://www.chosic.com/music-genre-finder/")
search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "search-word")))

for i in songs.index:
#for i in [38,39,40,41,42,45,46]: 
    to_split = songs.loc[i]
    track = to_split["track"]
    artist = to_split["artist"]
    
    
    # Check if track and artist is in existing here
    # Do here since it will need the songs.index and this way I won't have 2 of the same loops
    temp = existing[ existing["track"] == track]
    temp = temp[ temp["artist"] == artist]
    if len(temp)>0:
        continue
    # Also checking if it is in issue
    temp = issue[ issue["track"] == track]
    temp = temp[ temp["artist"] == artist]
    if len(temp)>0:
        continue
    
    
    search_bar.send_keys(track + " - " + artist)
    
    try:
        recomended_song = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@id='form-suggestions']/span[1]"))
        )
    except:
        issue.loc[len(issue)] = [i, artist, track,0]
        print("Something wrong at index: " + str(i))
        print("| Reseting website")
        driver.get("https://www.chosic.com/music-genre-finder/")
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "search-word")))
        continue
        
    if recomended_song.get_property("innerText") == "No songs found":
        issue.loc[len(issue)] = [i, artist, track,1]
        print("Something wrong at index: " + str(i))
        print("| Refreshing search bar")
        refresh()
        continue
    
    recomended_song.click()
    
    
    
    # Get genres off of webpage
    if is_genres():
        genre_parent = driver.find_element(By.XPATH, "//div[@class='spotify-result']/div[@class='pl-tags tagcloud']")
    else:
        issue.loc[len(issue)] = [i, artist, track,2]
        print("Something wrong at index: " + str(i))        
        print("| Reseting website")
        driver.get("https://www.chosic.com/music-genre-finder/")
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "search-word")))
        continue
             
    for genre in genre_parent.get_property("children"):
        categ.loc[len(categ.index)] = [artist, track, genre.get_property("innerText")]
    if len(genre_parent.get_property("children")) == 0:
        categ.loc[len(categ.index)] = [artist, track, "Unknown"]
    
    
    # Get song analysis off of webpage (popularitiy, happiness, ...)
    try:
        progress_bar = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='progressbars-div']"))
        )
    except:
        issue.loc[len(issue)] = [i, artist, track,3]
        print("Something wrong at index: " + str(i))
        print("| Lost information on Track Analysis")
        continue
    
    
    txt = progress_bar.get_property("textContent")

    stats = txt.split("/100")
    stats.remove(" ")

    # Create next line
    next_line = pd.Series(data=[artist,track], index=["artist", "track"])
    #Fill in the created line
    for stat in stats:
        (cat, cnt) = stat.strip().split(": ")
        next_line = pd.concat([next_line, pd.Series(data=[cnt], index=[cat])])

    tempo_parent = unique_element(driver, By.XPATH, "//div[@class='tempo-duration-first']") 
    for child in tempo_parent.get_property("children"):
        temp = child.get_property("textContent").strip()
        (cat, val) = temp.split(": ")
        next_line = pd.concat([next_line, pd.Series(data=[val], index=[cat])])

        
    # Combine the created line to the main df
    song_stats = pd.concat([song_stats, next_line.to_frame(1).T])
    
    
    
    # Display to show where we have gotten to
    if i % 50 == 0:
        print("Reached index " + str(i))
print("******************DONE*******************")

******************DONE*******************


In [10]:
def export_unique(df, filename):
    prev = pd.read_csv(filename)
    pd.concat([prev, df]).drop_duplicates().to_csv(filename, index=False)

In [11]:
# Exporting the Data
# I want to append unique values only to keep file size down

export_unique(issue, "Scraping Errors.csv")
export_unique(categ, "Data - Scraped Genres.csv")
export_unique(song_stats, "Data - Song Details.csv")

In [12]:
song_stats

,artist,track,Popularity,Happiness,Danceability,Energy,Acousticness,Instrumentalness,Liveness,Speechiness,Length,Tempo,Loudness
1,Barns Courtney,Kicks,60,62,64,83,13,0,29,7,3:15,95 bpm,-5 db
1,The Charlie Daniels Band,Sweet Louisiana - Live,14,71,41,91,20,3,91,6,4:42,107 bpm,-10 db
1,The Charlie Daniels Band,Long Haired Country Boy - Live,38,81,60,42,16,1,79,3,4:05,148 bpm,-14 db
1,The Charlie Daniels Band,Trudy - Live,12,63,49,91,40,0,89,5,4:56,100 bpm,-10 db
1,The Charlie Daniels Band,Cumberland Mountain Number Nine - Live,14,57,33,69,54,1,86,4,5:58,96 bpm,-12 db
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Greta Van Fleet,Age of Machine,60,11,22,81,0,1,8,5,6:53,136 bpm,-5 db
1,The HU,Sad But True,53,22,39,89,0,3,19,9,5:32,178 bpm,-5 db
1,The Mahones,Stars (Oscar Wilde),2,46,33,87,0,0,37,3,5:11,131 bpm,-5 db
1,Artificial Fear,The Mandalorian Theme,35,22,26,68,0,89,32,5,2:59,83 bpm,-6 db
